# Implements the functions for the net present value and levelized cost of energy

- NPV and NPV2 differ in how wholesale price is assumed to fluctuate. 
- In NPV it is an uncertain average number over the project lifetime. 
- In NPV2 it varies within given window during every year of the project. In NPV2 one could replace this randomness with the average wholesale price without large error.  

In [2]:
import numpy as np
import scipy.optimize as opt
#from scipy.optimize import brentq  #root finding

In [4]:
def NPV(construction_period,n,r,cf,It0,OMkwh,OMfixed,wholesale_price):
    prod=cf*1*365*24
    constructiont=int(round(construction_period))
    It=np.zeros(n+constructiont)
    #It[0]=It0
    Mt=np.ones(n+constructiont)*(OMkwh*prod+OMfixed)
    Et=np.ones(n+constructiont)*prod  #kwH
    if constructiont>0:  #if construction time zero, capex is the overnight cost         
        for x in range(constructiont):
            It[x]=It0/constructiont  #spread the investment evenly over the construction period
            Mt[x]=0   #No costs or production during construction
            Et[x]=0
    else:
        It[0]=It0    
    discounted_benefits=0.0
    discounted_costs=0.0
    for x in range(n+constructiont):
        t=x
        benefit=Et[x]*(wholesale_price/1000)/np.power((1+r),t)
        cost=(It[x]+Mt[x])/np.power((1+r),t)
        discounted_benefits=discounted_benefits+benefit
        discounted_costs=discounted_costs+cost
    return discounted_benefits-discounted_costs

#Assume that wholesale price simply fluctuates in time in the given range...i.e. permanently low wholesale price
def NPV2(construction_period,n,r,cf,It0,OMkwh,OMfixed,wholesalemin,wholesalemax):
    prod=cf*1*365*24
    constructiont=int(round(construction_period))
    It=np.zeros(n+constructiont)
    #It[0]=It0
    Mt=np.ones(n+constructiont)*(OMkwh*prod+OMfixed)
    Et=np.ones(n+constructiont)*prod  #kwH
    if constructiont>0.5:
        for x in range(constructiont):
            It[x]=It0/constructiont  #spread the investment evenly over the construction period
            Mt[x]=0   #No costs or production during construction
            Et[x]=0
    else:
        It[0]=It0
    discounted_benefits=0.0
    discounted_costs=0.0
    for x in range(n+constructiont):
        t=x
        wholesale_price=np.random.uniform(low=wholesalemin,high=wholesalemax)
        #wholesale_price=0.5*(wholesalemin+wholesalemax)
        benefit=Et[x]*(wholesale_price/1000)/np.power((1+r),t)
        cost=(It[x]+Mt[x])/np.power((1+r),t)
        discounted_benefits=discounted_benefits+benefit
        discounted_costs=discounted_costs+cost
    return discounted_benefits-discounted_costs

#Assume that wholesale price constant
def NPV3(construction_period,n,r,cf,It0,OMkwh,OMfixed,wholesale):
    prod=cf*1*365*24
    constructiont=int(round(construction_period))
    It=np.zeros(n+constructiont)
    #It[0]=It0
    Mt=np.ones(n+constructiont)*(OMkwh*prod+OMfixed)
    Et=np.ones(n+constructiont)*prod  #kwH
    if constructiont>0.5:
        for x in range(constructiont):
            It[x]=It0/constructiont  #spread the investment evenly over the construction period
            Mt[x]=0   #No costs or production during construction
            Et[x]=0
    else:
        It[0]=It0
    discounted_benefits=0.0
    discounted_costs=0.0
    for x in range(n+constructiont):
        t=x
        wholesale_price=wholesale
        #wholesale_price=0.5*(wholesalemin+wholesalemax)
        benefit=Et[x]*(wholesale_price/1000)/np.power((1+r),t)
        cost=(It[x]+Mt[x])/np.power((1+r),t)
        discounted_benefits=discounted_benefits+benefit
        discounted_costs=discounted_costs+cost
    return discounted_benefits-discounted_costs


def LCOE(construction_period,n,r,cf,It0,OMkwh,OMfixed):
    #Returns LCOE of the energy source in currency/kWh
    #Distribute investment evenly over the construction time...production starts after that
    prod=cf*1*365*24
    constructiont=int(round(construction_period))
    It=np.zeros(n+constructiont)
    #It[0]=It0
    Mt=np.ones(n+constructiont)*(OMkwh*prod+OMfixed)
    Et=np.ones(n+constructiont)*prod  #kwH
    if constructiont>0.5:
        for x in range(constructiont):
            It[x]=It0/constructiont  #spread the investment evenly over the construction period
            Mt[x]=0   #No costs or production during construction
            Et[x]=0
    else:
        It[0]=It0
        
    discounted_costs=0.0
    discounted_benefits=0.0
    for x in range(n+constructiont):
        t=x
        benefit=Et[x]/np.power((1+r),t)
        cost=(It[x]+Mt[x])/np.power((1+r),t)        
        discounted_benefits=discounted_benefits+benefit
        discounted_costs=discounted_costs+cost
    LCOEval=discounted_costs/discounted_benefits
    return LCOEval

#def NPV3(construction_period,n,r,cf,It0,OMkwh,OMfixed,wholesale):
#Find the discount rate at which NPV is zero
def IRR(construction_period,n,r,cf,It0,OMkwh,OMfixed,wholesale):
    #Root of NPV3
    a=-0.99
    b=0.5
    lower=NPV3(construction_period,n,a,cf,It0,OMkwh,OMfixed,wholesale)
    upper=NPV3(construction_period,n,b,cf,It0,OMkwh,OMfixed,wholesale)
    if lower*upper<0.0:
        r=opt.brentq(lambda discount: NPV3(construction_period,n,discount,cf,It0,OMkwh,OMfixed,wholesale),a,b)
        #r=brentq(NPV3, a, b, args=(), xtol=2e-6,rtol=1.0e-6, maxiter=50,full_output=False, disp=True
    else:
        r=-2.0
    return r